### Step 1: Install dependencies for model evaluation 🔽

In [1]:
!apt-get install -y \
    libgl1-mesa-dev \
    libgl1-mesa-glx \
    libglew-dev \
    libosmesa6-dev \
    software-properties-common \
    patchelf \
    xvfb

zsh:1: command not found: apt-get


### Step 2: Install and import the packages 📦



In [2]:
!pip install free-mujoco-py
!pip install transformers[torch]
!pip install transformers
!pip install datasets
!pip install imageio-ffmpeg

!pip install colabgymrender==1.0.2
!pip install xvfbwrapper
!pip install imageio==2.4.1
!pip install imageio-ffmpeg
!pip install huggingface_hub
!pip install accelerate -U


  Using cached imageio-2.31.6-py3-none-any.whl (313 kB)
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
zsh:1: no matches found: transformers[torch]

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
  Using cached imageio-2.4.1.tar.gz (3.3 MB)
  Preparing metadata

In [3]:
import os
import random
from dataclasses import dataclass

import numpy as np
import torch
from datasets import load_dataset
from transformers import Trainer, TrainingArguments
from dt.configuration_decision_transformer import DecisionTransformerConfig
from dt.modeling_decision_transformer import DecisionTransformerModel

/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 3: Loading the dataset from the 🤗 Hub and instantiating the model

We host a number of Offline RL Datasets on the hub. Today we will be training with the halfcheetah “expert” dataset, hosted here on hub.

First we need to import the load_dataset function from the 🤗 datasets package and download the dataset to our machine.

In [4]:
os.environ["WANDB_DISABLED"] = "true" # we diable weights and biases logging for this tutorial
dataset = load_dataset("edbeeching/decision_transformer_gym_replay", "halfcheetah-expert-v2")


In [5]:
len(dataset['train'])
dataset['train'][0].keys()

dict_keys(['observations', 'actions', 'rewards', 'dones'])

### Step 4: Defining a custom DataCollator for the transformers Trainer class

In [6]:
@dataclass
class DecisionTransformerGymDataCollator:
    return_tensors: str = "pt"
    max_len: int = 20 #subsets of the episode we use for training
    state_dim: int = 17  # size of state space
    act_dim: int = 6  # size of action space
    max_ep_len: int = 1000 # max episode length in the dataset
    scale: float = 1000.0  # normalization of rewards/returns
    state_mean: np.array = None  # to store state means
    state_std: np.array = None  # to store state stds
    p_sample: np.array = None  # a distribution to take account trajectory lengths
    n_traj: int = 0 # to store the number of trajectories in the dataset

    def __init__(self, dataset) -> None:
        self.act_dim = len(dataset[0]["actions"][0])
        self.state_dim = len(dataset[0]["observations"][0])
        self.dataset = dataset
        # calculate dataset stats for normalization of states
        states = []
        traj_lens = []
        for obs in dataset["observations"]:
            states.extend(obs)
            traj_lens.append(len(obs))
        self.n_traj = len(traj_lens)
        states = np.vstack(states)
        self.state_mean, self.state_std = np.mean(states, axis=0), np.std(states, axis=0) + 1e-6
        
        traj_lens = np.array(traj_lens)
        self.p_sample = traj_lens / sum(traj_lens)

    def _discount_cumsum(self, x, gamma):
        discount_cumsum = np.zeros_like(x)
        discount_cumsum[-1] = x[-1]
        for t in reversed(range(x.shape[0] - 1)):
            discount_cumsum[t] = x[t] + gamma * discount_cumsum[t + 1]
        return discount_cumsum

    def __call__(self, features):
        batch_size = len(features)
        # this is a bit of a hack to be able to sample of a non-uniform distribution
        batch_inds = np.random.choice(
            np.arange(self.n_traj),
            size=batch_size,
            replace=True,
            p=self.p_sample,  # reweights so we sample according to timesteps
        )
        # a batch of dataset features
        s, a, r, d, rtg, timesteps, mask = [], [], [], [], [], [], []
        
        for ind in batch_inds:
            # for feature in features:
            feature = self.dataset[int(ind)]
            si = random.randint(0, len(feature["rewards"]) - 1)

            # get sequences from dataset
            s.append(np.array(feature["observations"][si : si + self.max_len]).reshape(1, -1, self.state_dim))
            a.append(np.array(feature["actions"][si : si + self.max_len]).reshape(1, -1, self.act_dim))
            r.append(np.array(feature["rewards"][si : si + self.max_len]).reshape(1, -1, 1))

            d.append(np.array(feature["dones"][si : si + self.max_len]).reshape(1, -1))
            timesteps.append(np.arange(si, si + s[-1].shape[1]).reshape(1, -1))
            timesteps[-1][timesteps[-1] >= self.max_ep_len] = self.max_ep_len - 1  # padding cutoff
            rtg.append(
                self._discount_cumsum(np.array(feature["rewards"][si:]), gamma=1.0)[
                    : s[-1].shape[1]   # TODO check the +1 removed here
                ].reshape(1, -1, 1)
            )
            if rtg[-1].shape[1] < s[-1].shape[1]:
                print("if true")
                rtg[-1] = np.concatenate([rtg[-1], np.zeros((1, 1, 1))], axis=1)

            # padding and state + reward normalization
            tlen = s[-1].shape[1]
            s[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, self.state_dim)), s[-1]], axis=1)
            s[-1] = (s[-1] - self.state_mean) / self.state_std
            a[-1] = np.concatenate(
                [np.ones((1, self.max_len - tlen, self.act_dim)) * -10.0, a[-1]],
                axis=1,
            )
            r[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), r[-1]], axis=1)
            d[-1] = np.concatenate([np.ones((1, self.max_len - tlen)) * 2, d[-1]], axis=1)
            rtg[-1] = np.concatenate([np.zeros((1, self.max_len - tlen, 1)), rtg[-1]], axis=1) / self.scale
            timesteps[-1] = np.concatenate([np.zeros((1, self.max_len - tlen)), timesteps[-1]], axis=1)
            mask.append(np.concatenate([np.zeros((1, self.max_len - tlen)), np.ones((1, tlen))], axis=1))

        s = torch.from_numpy(np.concatenate(s, axis=0)).float()
        a = torch.from_numpy(np.concatenate(a, axis=0)).float()
        r = torch.from_numpy(np.concatenate(r, axis=0)).float()
        d = torch.from_numpy(np.concatenate(d, axis=0))
        rtg = torch.from_numpy(np.concatenate(rtg, axis=0)).float()
        timesteps = torch.from_numpy(np.concatenate(timesteps, axis=0)).long()
        mask = torch.from_numpy(np.concatenate(mask, axis=0)).float()

        return {
            "states": s,
            "actions": a,
            "rewards": r,
            "returns_to_go": rtg,
            "timesteps": timesteps,
            "attention_mask": mask,
        }

### Step 5: Extending the Decision Transformer Model to include a loss function

In order to train the model with the 🤗 trainer class, we first need to ensure the dictionary it returns contains a loss, in this case L-2 norm of the models action predictions and the targets.

In [7]:
class TrainableDT(DecisionTransformerModel):
    def __init__(self, config):
        super().__init__(config)

    def forward(self, **kwargs):
        output = super().forward(**kwargs)
        # add the DT loss
        action_preds = output[1]
        action_targets = kwargs["actions"]
        attention_mask = kwargs["attention_mask"]
        act_dim = action_preds.shape[2]
        action_preds = action_preds.reshape(-1, act_dim)[attention_mask.reshape(-1) > 0]
        action_targets = action_targets.reshape(-1, act_dim)[attention_mask.reshape(-1) > 0]
        
        loss = torch.mean((action_preds - action_targets) ** 2)

        return {"loss": loss}

    def original_forward(self, **kwargs):
        return super().forward(**kwargs)

In [8]:
collator = DecisionTransformerGymDataCollator(dataset["train"])

config = DecisionTransformerConfig(state_dim=collator.state_dim, act_dim=collator.act_dim)
model = TrainableDT(config)

### Step 6: Defining the training hyperparameters and training the model
Here, we define the training hyperparameters and our Trainer class that we'll use to train our Decision Transformer model.

This step takes about an hour, so you may leave it running. Note the authors train for at least 3 hours, so the results presented here are not as performant as the models hosted on the 🤗 hub.

In [9]:
training_args = TrainingArguments(
    output_dir="output/",
    remove_unused_columns=False,
    num_train_epochs=120,
    per_device_train_batch_size=64,
    learning_rate=1e-4,
    weight_decay=1e-4,
    warmup_ratio=0.1,
    optim="adamw_torch",
    max_grad_norm=0.25,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    data_collator=collator,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
 10%|█         | 194/1920 [10:46<1:35:43,  3.33s/it]

KeyboardInterrupt: 

### Step 7: Visualize the performance of the agent

In [10]:
import mujoco_py
import gym

from colabgymrender.recorder import Recorder

Compiling /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/cymj.pyx because it changed.
[1/1] Cythonizing /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/cymj.pyx
running build_ext
building 'mujoco_py.cymj' extension
creating /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_macextensionbuilder
creating /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_macextensionbuilder/temp.macosx-13.1-x86_64-cpython-310
creating /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_macextensionbuilder/temp.macosx-13.1-x86_64-cpython-310/Users
creating /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_macextensionbuilder/temp.macosx-13.1-x86_64-cpython-310/Users/jacob
creating /Users/jacob/.pyenv/versions/3.10.12/lib/python3.

gcc-9: warning: could not understand version 13.1


/usr/local/bin/gcc-9 -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -DONMAC -I/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py -I/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/binaries/macos/mujoco210/include -I/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/numpy/core/include -I/Users/jacob/.pyenv/versions/3.10.12/include/python3.10 -c /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/gl/dummyshim.c -o /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_macextensionbuilder/temp.macosx-13.1-x86_64-cpython-310/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/gl/dummyshim.o -fopenmp -w
creating /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_macextensionbuilder/lib.macosx-13.1-x86_64-cpython-310
creating /Users/jacob

gcc-9: warning: could not understand version 13.1
gcc-9: warning: could not understand version 13.1
ld: -rpath can only be used when targeting Mac OS X 10.5 or later
collect2: error: ld returned 1 exit status


LinkError: command '/usr/local/bin/gcc-9' failed with exit code 1

In [11]:
# Function that gets an action from the model using autoregressive prediction with a window of the previous 20 timesteps.
def get_action(model, states, actions, rewards, returns_to_go, timesteps):
    # This implementation does not condition on past rewards

    states = states.reshape(1, -1, model.config.state_dim)
    actions = actions.reshape(1, -1, model.config.act_dim)
    returns_to_go = returns_to_go.reshape(1, -1, 1)
    timesteps = timesteps.reshape(1, -1)

    states = states[:, -model.config.max_length :]
    actions = actions[:, -model.config.max_length :]
    returns_to_go = returns_to_go[:, -model.config.max_length :]
    timesteps = timesteps[:, -model.config.max_length :]
    padding = model.config.max_length - states.shape[1]
    # pad all tokens to sequence length
    attention_mask = torch.cat([torch.zeros(padding), torch.ones(states.shape[1])])
    attention_mask = attention_mask.to(dtype=torch.long).reshape(1, -1)
    states = torch.cat([torch.zeros((1, padding, model.config.state_dim)), states], dim=1).float()
    actions = torch.cat([torch.zeros((1, padding, model.config.act_dim)), actions], dim=1).float()
    returns_to_go = torch.cat([torch.zeros((1, padding, 1)), returns_to_go], dim=1).float()
    timesteps = torch.cat([torch.zeros((1, padding), dtype=torch.long), timesteps], dim=1)

    state_preds, action_preds, return_preds = model.original_forward(
        states=states,
        actions=actions,
        rewards=rewards,
        returns_to_go=returns_to_go,
        timesteps=timesteps,
        attention_mask=attention_mask,
        return_dict=False,
    )

    return action_preds[0, -1]

In [14]:
import gymnasium as gym
# build the environment
directory = './video'
model = model.to("cpu")
env = gym.make("HalfCheetah-v3")
env = Recorder(env, directory, fps=30)
max_ep_len = 1000
device = "cpu"
scale = 1000.0  # normalization for rewards/returns
TARGET_RETURN = 12000 / scale  # evaluation is conditioned on a return of 12000, scaled accordingly

state_mean = collator.state_mean.astype(np.float32)
state_std = collator.state_std.astype(np.float32)
print(state_mean)

state_dim = env.observation_space.shape[0]
act_dim = env.action_space.shape[0]
# Create the decision transformer model

state_mean = torch.from_numpy(state_mean).to(device=device)
state_std = torch.from_numpy(state_std).to(device=device)


running build_ext
building 'mujoco_py.cymj' extension
/usr/local/bin/gcc-9 -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -DONMAC -I/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py -I/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/binaries/macos/mujoco210/include -I/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/numpy/core/include -I/Users/jacob/.pyenv/versions/3.10.12/include/python3.10 -c /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/cymj.c -o /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_macextensionbuilder/temp.macosx-13.1-x86_64-cpython-310/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/cymj.o -fopenmp -w


/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment HalfCheetah-v3 is out of date. You should consider upgrading to version `v4`.
  logger.deprecation(
gcc-9: warning: could not understand version 13.1


/usr/local/bin/gcc-9 -Wno-unused-result -Wsign-compare -Wunreachable-code -DNDEBUG -g -fwrapv -O3 -Wall -DONMAC -I/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py -I/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/binaries/macos/mujoco210/include -I/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/numpy/core/include -I/Users/jacob/.pyenv/versions/3.10.12/include/python3.10 -c /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/gl/dummyshim.c -o /Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/generated/_pyxbld_2.0.2.13_310_macextensionbuilder/temp.macosx-13.1-x86_64-cpython-310/Users/jacob/.pyenv/versions/3.10.12/lib/python3.10/site-packages/mujoco_py/gl/dummyshim.o -fopenmp -w
/usr/local/bin/gcc-9 -bundle -undefined dynamic_lookup -L/usr/local/opt/readline/lib -L/usr/local/opt/readline/lib -L/Users/jacob/.pyenv/versions/3.10.12/lib -Wl,-rpath,/Users/jacob/.pyenv/ver

gcc-9: warning: could not understand version 13.1
gcc-9: warning: could not understand version 13.1
ld: -rpath can only be used when targeting Mac OS X 10.5 or later
collect2: error: ld returned 1 exit status


LinkError: command '/usr/local/bin/gcc-9' failed with exit code 1

In [ ]:
# Interact with the environment and create a video
episode_return, episode_length = 0, 0
state = env.reset()
target_return = torch.tensor(TARGET_RETURN, device=device, dtype=torch.float32).reshape(1, 1)
states = torch.from_numpy(state).reshape(1, state_dim).to(device=device, dtype=torch.float32)
actions = torch.zeros((0, act_dim), device=device, dtype=torch.float32)
rewards = torch.zeros(0, device=device, dtype=torch.float32)

timesteps = torch.tensor(0, device=device, dtype=torch.long).reshape(1, 1)
for t in range(max_ep_len):
    actions = torch.cat([actions, torch.zeros((1, act_dim), device=device)], dim=0)
    rewards = torch.cat([rewards, torch.zeros(1, device=device)])

    action = get_action(
        model,
        (states - state_mean) / state_std,
        actions,
        rewards,
        target_return,
        timesteps,
    )
    actions[-1] = action
    action = action.detach().cpu().numpy()

    state, reward, done, _ = env.step(action)

    cur_state = torch.from_numpy(state).to(device=device).reshape(1, state_dim)
    states = torch.cat([states, cur_state], dim=0)
    rewards[-1] = reward

    pred_return = target_return[0, -1] - (reward / scale)
    target_return = torch.cat([target_return, pred_return.reshape(1, 1)], dim=1)
    timesteps = torch.cat([timesteps, torch.ones((1, 1), device=device, dtype=torch.long) * (t + 1)], dim=1)

    episode_return += reward
    episode_length += 1

    if done:
        break


In [ ]:
# Play the video
env.play()
# If you want to convert the video:
# !ffmpeg -i {your_video} -vcodec h264 replay.mp4 

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
trainer.push_to_hub()